In [3]:
system("conda install r-sp r-phyloregion=1.0.8 r-ape r-matrix r-sf r-raster r-dplyr")

In [4]:
install.packages("BiocManager")
BiocManager::install("SparseArray")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.18 (BiocManager 1.30.25), R 4.3.3 (2024-02-29)

Installing package(s) 'BiocVersion', 'SparseArray'

also installing the dependencies ‘zlibbioc’, ‘BiocGenerics’, ‘MatrixGenerics’, ‘S4Vectors’, ‘S4Arrays’, ‘matrixStats’, ‘IRanges’, ‘XVector’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'abind', 'caret', 'cli', 'diffobj', 'gert', 'gower', 'httpuv',
  'igraph', 'RCurl', 'RODBC', 'scales', 'systemfonts', 'units', 'XML'



In [5]:
library(phyloregion)
library(ape)
library(Matrix)
library(SparseArray)
library(sf)
library(sp)
library(raster)
library(dplyr)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:phyloregion’:

    counts


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsum

In [6]:
args = c("galaxy_inputs/matrix/matrix_file.csv", "galaxy_inputs/tree/tree_file.txt.txt", "galaxy_inputs/grid/grid_aq_3_PF.shp")

In [24]:
save_sf <- function(){
  #st_write(phyloreg_sf[,-(3:5)], paste0(tempdir(), "/", "output.shp"), delete_layer = TRUE)
  write_sf(phyloreg_sf, "output.shp", delete_layer = TRUE)
}

In [8]:
comm_matrix <- readSparseCSV(args[1])
comm_matrix <- as(comm_matrix,"dgCMatrix")

In [9]:
comm_tree <- read.tree(args[2])

In [15]:
grid <- read_sf(args[3])
# manipulation temporaire ( a retirer quand usage avec script CRSConverter (outils 1))
prj_laea     <- "+proj=laea +lat_0=-90 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs" 
grid <- st_transform(grid, crs = prj_laea)

In [16]:
phylo_beta <- phylobeta(comm_matrix, comm_tree, index.family = "sorensen")

In [17]:
select_linkage(phylo_beta[[1]])


A good clustering algorithm for the distance matrix is:
 UPGMA with cophenetic correlation = 0.5754278 



UPGMA    Single  Complete    ward.D   ward.D2     WPGMA     WPGMC     UPGMC 
0.5754278 0.1692290 0.4560127 0.4397381 0.4202388 0.4245704 0.4625974 0.5501966

In [18]:
optim <- optimal_phyloregion(phylo_beta[[3]], k = 30)

In [19]:
grid_sp <- as_Spatial(grid)

In [20]:
y <- phyloregion(phylo_beta[[3]], pol = grid_sp, k = 8, method = "average")

Warning message in mixcolor(seq(0, 199)/199, polarLUV(70, 50, 30), polarLUV(70, :
“convex combination of colors in polar coordinates (polarLUV) may not be appropriate”
Warning message in mixcolor(seq(0, 199)/199, polarLUV(70, 50, 300), polarLUV(70, :
“convex combination of colors in polar coordinates (polarLUV) may not be appropriate”


In [21]:
phyloreg_sf <- y$pol
  st_crs(phyloreg_sf)
  #plot(phyloreg_sf)
  phyloreg_sf <- st_as_sf(phyloreg_sf, crs = "+proj=laea +lat_0=-90 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

Coordinate Reference System:
  User input: +proj=laea +lat_0=-90 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs 
  wkt:
PROJCRS["unknown",
    BASEGEOGCRS["unknown",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]],
            ID["EPSG",6326]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8901]]],
    CONVERSION["unknown",
        METHOD["Lambert Azimuthal Equal Area",
            ID["EPSG",9820]],
        PARAMETER["Latitude of natural origin",-90,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["False easting",0,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]],
        PARAMETER["False northing",0,
            LENGTHUNIT["metre",1],
 

In [22]:
names(phyloreg_sf)[3:8] <- c("R_val", "G_val", "B_val", "r_code", "g_code", "b_code")

In [25]:
save_sf()